# Load required modules

In [ ]:
import psycopg2  # handles database connection to postgres db
import pandas.io.sql as sqlio  # we'll use this for improved query presentation

# Connect to the relational database

In [ ]:
# get from templates folder

## Let us take a look at isolation from another notebook
### First, check the table "kunde" while the other transaction is running.

In [ ]:
# here is the second transaction that is not aware of that what happens in the first one
sql = "SELECT * FROM kunde"

dat = sqlio.read_sql_query(sql, conn)
dat # the new customer "Jansen" should not be visible while the other transaction is running

### Second, manipulate the tables "kunde"  and "auftrag" while the other transaction is running - this leads to an inconsistency in the other transaction

In [ ]:
from time import sleep  # we will use this to start "waiting"

insert_into_customer = "INSERT INTO kunde VALUES (4, 'Schulz')"
insert_into_orders = "INSERT INTO auftrag VALUES (20, 4, 'Banane')"  # NOTE: here, we use the order_id from the second insert

try:
    cur = conn.cursor()  # retrieving a database cursor starts the transaction
    # the following two statements are executed in an all-or-none-proposition
    cur.execute(insert_into_customer)
    # we do not sleep here - so this transaction should be finishing first
    cur.execute(insert_into_orders)
    conn.commit()    
except Exception as e:
    print("Error inserting into table: {}".format(e))
    conn.rollback()  # in case the transaction cannot be commit, we want to roll back to the previous state (CONSISTENCY)


# Select all elements from table auftrag and kunde (joined)
sql = """
SELECT * FROM kunde LEFT JOIN auftrag ON auftrag.auftrag_kunde_id=kunde.kunde_id
"""

dat = sqlio.read_sql_query(sql, conn)
dat # the new customer "Jansen" should not be added to the database

### This cell is used to repeat the case

In [ ]:
try:
    cur = conn.cursor()
    cur.execute("DELETE FROM auftrag WHERE auftrag_kunde_id=4")
    cur.execute("DELETE FROM kunde WHERE kunde_id=4")    
    conn.commit()    
except Exception as e:
    print("Error deleting from table: {}".format(e))
    conn.rollback()  # in case the transaction cannot be commit, we want to roll back to the previous state (CONSISTENCY)